In [110]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

df = pd.read_csv('extraksiSidikJari.csv')
len_row, len_col = df.shape
df.fillna(df.mode().iloc[0], inplace=True) # Replace NaN values with mode of corresponding column
# use the factorize() function to convert the column of strings to integers
non_int_columns = df.select_dtypes(include=['object']).columns.tolist()
    
# Delete duplicate rows
df = df.drop_duplicates()

In [71]:
print(len_row, len_col)

100 37


In [111]:
label_mapping = {1: 0, 2: 1, 3: 2, 4: 3, 5: 4}

df['Target'] = df['Target'].replace(label_mapping)

In [3]:
label_mapping = {'A': 0, 'B': 1, 'C': 2, 'D': 3}

df['class'] = df['class'].replace(label_mapping)

label_mapping = {'M': 1, 'F': 2}

df['gender'] = df['gender'].replace(label_mapping)

In [112]:
df = df.iloc[:, 1:]

In [144]:
df = df.drop(['stdevy135','korelasi135'], axis=1)

In [113]:
df.head(20)

,mu,varians_n,deviasi,skewness,energi,entropi,smoothness,asm0,idm0,stdevy0,...,entropi90,energy90,asm135,idm135,stdevy135,korelasi135,kontras135,entropi135,energy135,Target
0,145.878641,0.028206,42.826449,2.317846,0.114321,5.502868,0.027432,0.057637,0.332372,9836.113792,...,6.289958,0.053554,0.040550,0.260301,9977.57043,0.000055,12312.538867,6.502044,0.040550,0
1,157.544397,0.031288,45.105789,0.728292,0.186542,4.746257,0.030339,0.138548,0.465802,10113.634109,...,5.233408,0.129908,0.122249,0.414700,9977.57043,0.000055,9158.559511,5.368045,0.122249,1
2,122.033879,0.023271,38.899822,6.477194,0.107866,5.576895,0.022742,0.042778,0.307723,10333.147300,...,6.473490,0.039466,0.029825,0.246460,9977.57043,0.000055,12070.446598,6.619654,0.029825,2
3,127.809567,0.025716,40.892211,4.815160,0.090897,5.841717,0.025071,0.042098,0.306074,9775.880965,...,6.746558,0.033565,0.032915,0.256447,9977.57043,0.000055,9972.608595,6.751549,0.032915,3
4,147.561590,0.029471,43.776078,1.830432,0.109280,5.582667,0.028627,0.059552,0.352693,9695.617897,...,6.583078,0.040549,0.028748,0.219472,9977.57043,0.000055,15205.186223,6.692705,0.028748,4
5,135.801477,0.026701,41.668019,4.343540,0.121439,5.376401,0.026006,0.063202,0.364949,10556.511325,...,6.209292,0.052712,0.060134,0.348529,9977.57043,0.000055,7235.328418,6.075022,0.060134,0
6,128.013552,0.021960,37.787894,7.365299,0.192115,4.433121,0.021488,0.131541,0.524057,11971.675810,...,4.965353,0.116167,0.102134,0.432452,9977.57043,0.000055,9552.090794,5.099214,0.102134,1
7,111.282464,0.018275,34.472269,8.771378,0.128111,5.335125,0.017947,0.053619,0.351741,10523.318240,...,6.243224,0.044804,0.043011,0.307010,9977.57043,0.000055,9259.973878,6.236152,0.043011,2
8,100.671521,0.012993,29.066207,12.502323,0.205661,4.360316,0.012826,0.119169,0.510541,11424.900293,...,4.741828,0.124358,0.087048,0.426757,9977.57043,0.000055,8502.140931,5.070821,0.087048,3
9,132.560882,0.024039,39.536401,5.746635,0.151363,4.934828,0.023475,0.090183,0.446949,11380.965356,...,5.926000,0.056210,0.072079,0.377070,9977.57043,0.000055,8867.783653,5.695982,0.072079,4


In [145]:
# Membaca data
X = np.array(df.iloc[:, :-1])
y = np.array(df.iloc[:, -1])

# Create a StandardScaler object
scaler = StandardScaler()

# Fit the scaler to the dataset
scaler.fit(X)

# Transform the dataset using the scaler
X_std = scaler.transform(X)

In [146]:
%run func.ipynb

In [147]:
def calculate_F1(X, y):
    c = len(np.unique(y))  # Number of classes
    means = np.zeros(c)  # Array to store means of each class
    variances = np.zeros(c)  # Array to store variances of each class
    proportions = np.zeros(c)  # Array to store proportions of each class
    
    # Calculate means, variances, and proportions of each class
    for i in range(c):
        means[i] = np.mean(X[y == i])
        
        variances[i] = np.var(X[y == i])
        
        proportions[i] = np.sum(y == i) / len(y)
    numerator = 0.0
    denominator = 0.0

    # Calculate numerator and denominator of F1
    for i in range(c):
        for j in range(c):
            if i != j:
                numerator += proportions[i] * proportions[j] * (means[i] - means[j]) ** 2

        denominator += proportions[i] * variances[i]
    F1 = numerator / denominator
    return F1

# Calculating the F1 score
F1_score = calculate_F1(X_std, y)

print("Maximum Fisher's discriminant ratio (F1):", F1_score)

Maximum Fisher's discriminant ratio (F1): 0.030388699374932642


In [148]:
# Assuming X_std and y are defined
feature_weights = reliefF(X_std, y)

In [149]:
combines = {'Features': df.iloc[:, :-1].columns, 'Weights': feature_weights}
weight_f = pd.DataFrame(combines)
weight_f = weight_f.sort_values('Weights', ascending=False)

In [150]:
from sklearn.preprocessing import MinMaxScaler

# Normalize the F1 values using Min-Max scaling
weight_f_norm = weight_f
scaler = MinMaxScaler()
weight_f_norm['Normalized F1'] = scaler.fit_transform(weight_f[['Weights']])

In [151]:
weight_f_norm.head(100)

,Features,Weights,Normalized F1
25,kontras90,0.165481,1.000000
11,kontras0,0.154483,0.896744
3,skewness,0.144704,0.804926
30,kontras135,0.136696,0.729748
18,kontras45,0.135497,0.718487
9,stdevy0,0.133176,0.696699
16,stdevy45,0.130555,0.672086
23,stdevy90,0.129416,0.661396
24,korelasi90,0.127185,0.640455
10,korelasi0,0.126929,0.638049
